A monoid is a set $M$ together with a binary operation:
$$\ast: M\times M\to M$$
i.e.
$$\ast:(g,h)\to g\ast h$$
and a "unit" element $1_M$

The binary operation must be associative:

$$g\ast (h\ast k) = (g\ast h)\ast k$$

The unit element must "do nothing":
$$ 1_M\ast g = g = g\ast 1_M$$

## Examples

1. Integers/Rationals/Natural Numbers/Real Numbers (under addition)
2. Integers/Rationals/Natural Numbers/Real Numbers (under multiplication)
3. Lists (under concatenation)
4. {True, False} (under "or" operation)
5. {True, False} (under "and" operation)
6. Sets (under union)
7. Subsets of a "Global" set (under intersection)
8. All Functions $f:A->A$ for a fixed set $A$ form a monoid under composition.

## Code Examples:

In [2]:
class Monoid:
    def __mul__(self, other):
        raise NotImplementedError
    
    @staticmethod
    def unit(self):
        raise NotImplementedError
        

In [3]:
class IntAdd(Monoid):
    def __init__(self, n: int):
        self.n = n
    
    def __mul__(self, other):
        return IntAdd(self.n + other.n)
    
    @staticmethod
    def unit(self):
        return IntAdd(0)
    
    def __repr__(self):
        return str(self.n)
    
class IntMult(Monoid):
    def __init__(self, n: int):
        self.n = n
    
    def __mul__(self, other):
        return IntMult(self.n * other.n)
    
    @staticmethod
    def unit(self):
        return IntMult(1)
    
    def __repr__(self):
        return str(self.n)
    
    
class ListMonoid(Monoid):
    def __init__(self, *elems):
        self.elems = elems
    
    def __mul__(self, other):
        return ListMonoid(*(self.elems + other.elems))
    
    @staticmethod
    def unit(self):
        return ListMonoid()
    
    def __repr__(self):
        return str(self.elems)
    

## Why it matters:

Monoids give maps 

```List[A] -> A```

Example: 
    ```reduce: (f: A x B -> A), (b_s: List[B])```
    currying turns `f` into `f_c: B -> (A -> A)`
    applying `f_c` to each element of `b_s` yields a `List[(A->A)]`
    aggregating (composing the functions) yields a single function `reduce(f, b_s) = (A->A)`

## Example (FiberedMonoids):

In [4]:
class FiberedMonoid(Monoid):
    def __init__(self, **vals: Monoid):
        self.vals = vals
    
    def __mul__(self, other):
        all_keys = set(self.vals.keys()) | set(other.vals.keys())
        def helper_mul(key, d1, d2):
            if key in d1 and key in d2:
                return d1[key] * d2[key]
            if key in d1:
                return d1[key]
            if key in d2:
                return d2[key]
        return FiberedMonoid(**{
            key: helper_mul(key, self.vals, other.vals)
            for key in all_keys
        })
    
    def __repr__(self):
        return repr(self.vals)

In [10]:
adi_s_ballot = FiberedMonoid(
    trump=IntAdd(3), 
    clinton=IntAdd(8), 
    favorite_foods=ListMonoid("pizza", "mac and cheese"),
    location_preferences=FiberedMonoid(
        sf=IntAdd(2),
        oakland=IntAdd(1)
    )
)
david_s_ballot = FiberedMonoid(
    trump=IntAdd(3), 
    clinton=IntAdd(12), 
    favorite_foods=ListMonoid("candy", "water"),
    location_preferences=FiberedMonoid(
        sf=IntAdd(2),
        oakland=IntAdd(3)
    )
)


In [11]:
adi_s_ballot * david_s_ballot

{'favorite_foods': ('pizza', 'mac and cheese', 'candy', 'water'), 'location_preferences': {'oakland': 4, 'sf': 4}, 'clinton': 20, 'trump': 6}

In [13]:
user_1 = FiberedMonoid(stemi=IntAdd(2), myocardial_infarction=IntAdd(1))
user_2 = FiberedMonoid(nstemi=IntAdd(2), myocardial_infarction=IntAdd(1))
user_3 = FiberedMonoid(heart_attack=IntAdd(2), myocardial_infarction=IntAdd(1))
user_4 = FiberedMonoid(heart_attach=IntAdd(2), myocardial_infarction=IntAdd(1))
user_1 * user_2 * user_3 * user_4

{'myocardial_infarction': 4, 'heart_attack': 2, 'stemi': 2, 'nstemi': 2, 'heart_attach': 2}

In [15]:
user_1 = FiberedMonoid(
    stemi=FiberedMonoid(
        vote=IntAdd(2),
        users=ListMonoid(1)
    ),
    myocardial_infarction=FiberedMonoid(
        vote=IntAdd(1),
        users=ListMonoid(1)
    ),
)
user_2 = FiberedMonoid(
    nstemi=FiberedMonoid(
        vote=IntAdd(2),
        users=ListMonoid(2)
    ), 
    myocardial_infarction=FiberedMonoid(
        vote=IntAdd(1),
        users=ListMonoid(2)
    ),
)
user_3 = FiberedMonoid(
    heart_attack=FiberedMonoid(
        vote=IntAdd(2),
        users=ListMonoid(3)
    ), 
    myocardial_infarction=FiberedMonoid(
        vote=IntAdd(1),
        users=ListMonoid(3)
    ),
)
user_4 = FiberedMonoid(
    heart_attach=FiberedMonoid(
        vote=IntAdd(2),
        users=ListMonoid(4)
    ), 
    myocardial_infarction=FiberedMonoid(
        vote=IntAdd(1),
        users=ListMonoid(4)
    ),
)
user_1 * user_2 * user_3 * user_4

{'myocardial_infarction': {'users': (1, 2, 3, 4), 'vote': 4}, 'heart_attack': {'vote': 2, 'users': (3,)}, 'stemi': {'vote': 2, 'users': (1,)}, 'nstemi': {'vote': 2, 'users': (2,)}, 'heart_attach': {'vote': 2, 'users': (4,)}}

In [ ]:
user_1 = FiberedMonoid(plan=Instructions("GetTheEmbedding"))
user_2 = FiberedMonoid(plan=Instructions("GetTheEmbedding"))
user_3 = FiberedMonoid(plan=Instructions("GetTheEmbedding"))
user_4 = FiberedMonoid(plan=Instructions("GetTheEmbedding"))
user_1 * user_2 * user_3 * user_4